# Gene expression profiles to identify cancer types

## Importing Libraries

In [ ]:
#data manipulation
import pandas as pd
import numpy as np

#data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import os

#feature selection
from sklearn.feature_selection import mutual_info_classif

#preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

#Classification
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, accuracy_score, classification_report

## Reading the data file

In [ ]:
#read data from github?
# Define the base directory for the data files
# Define the raw URLs for the data files
base_dir = '../data/raw/'


# Construct the full paths
data_path = os.path.join(base_dir, 'data.csv')
labels_path = os.path.join(base_dir, 'labels.csv')

dt = pd.read_csv(data_path, index_col = 0)
lb = pd.read_csv(labels_path, index_col = 0)

## Data exploration and cleaning

In [ ]:
#merging the data with labels

data_frame = pd.merge(dt, lb, left_index=True, right_index=True)
print(data_frame.columns)
print (data_frame.shape)

#Rename last column as Cancer_Type
data_frame = data_frame.rename(columns={'Class': 'Cancer_Type'})
print(data_frame.columns)

In [ ]:
#Check for missing values - this summs the amount of null occurrences in the data set and safe the columns with null values in variable null
datanull = data_frame.isnull().sum() 
null = [i for i in datanull if i > 0]

print('The columns with missing values are:%d'%len(null))

In [ ]:
#Checking how many type of cancer we have
print(data_frame['Cancer_Type'].value_counts())

#Visualizing
data_frame['Cancer_Type'].value_counts().plot.bar(color='purple')

# Add labels and title
plt.xlabel('Cancer Types')
plt.ylabel('# samples per type')
plt.title('Amount of samples per cancer type')

In [ ]:
#Separating feature values from class (cancer type)
X = data_frame.iloc[:, :-1]  # Features (gene expression levels)
y = data_frame.iloc[:, -1]   # Target variable (last column)


## Encoding the lables 
Since the class (cancer type) is categorical we will need to convert (encode) them to numeric to be able to perform the analysis.

In [ ]:
#using LabelEncoder from sklearn 
label_encoder = LabelEncoder()
label_encoder.fit(y)
y_encoded = label_encoder.transform(y)
labels = label_encoder.classes_
classes = np.unique(y_encoded)
print(labels)
print(classes)

### Data spliting

This section will allow us to split the data into training and testing subsets. 

In [ ]:
#spliting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y_encoded,test_size=0.2, random_state=42)


### Data normalization

This will imporve the model performace, it will set the values to the same range.

In [ ]:
# Initialize MinMaxScaler
min_max_scaler = MinMaxScaler()

# Fit and transform the training data
X_train_norm = min_max_scaler.fit_transform(X_train)

# Transform the test data using the same scaler
X_test_norm = min_max_scaler.transform(X_test)

### Feature Selection

Selecting the top genes(features)

In [ ]:
mic_features = mutual_info_classif(X_train_norm,y_train)

In [ ]:
#arbitrarily selected top #
#you can modify the value and see how the performance of the model changes

n_features=50
selected_scores_indices=np.argsort(mic_features)[::-1][0:n_features]

In [ ]:
X_train_selected=X_train_norm[:,selected_scores_indices]
X_test_selected=X_test_norm[:,selected_scores_indices]

In [ ]:
X_train_selected.shape

In [ ]:
X_test_selected.shape

### Classification

For the purpose of this project we are doing a regression

In [ ]:
# Initialize linear regression model
model = LinearRegression()

# Fit the model on the training data
model.fit(X_train_selected, y_train)

# Predict on the test data
y_pred = model.predict(X_test_selected)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

 ### Visualization of the results
 
 #### Scatter Plot of Actual vs. Predicted Values
 Points clustered closely around the diagonal line indicate accurate predictions. Deviations from this line suggest where the model overpredicts or underpredicts.

In [ ]:
# Scatter plot of actual vs. predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color='blue', alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  # Diagonal line
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values')
plt.show()

#### Residual Plot

The residual plot shows the residuals (difference between actual and predicted values) against the predicted values.

Ideally, residuals should be randomly scattered around the horizontal line at y=0, without any clear pattern. Patterns could indicate heteroscedasticity or other issues with the model.

In [ ]:
# Residual plot
plt.figure(figsize=(8, 6))
plt.scatter(y_pred, y_test - y_pred, color='blue', alpha=0.5)
plt.axhline(y=0, color='k', linestyle='--', lw=2)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

#### Distribution of Residuals
Visualizing the distribution of residuals can provide insights into the model's performance. Residuals ideally should be normally distributed around 0. Skewness or heavy tails indicate bias in the model.

In [ ]:
# Distribution plot of residuals
plt.figure(figsize=(8, 6))
residuals = y_test - y_pred
plt.hist(residuals, bins=20, color='blue', alpha=0.5)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Distribution of Residuals')
plt.show()

#### Coefficient Plot

Show which features are most influential in predicting the target variable. Larger coefficients (positive or negative) indicate stronger influence of the corresponding feature on the target variable.

In [ ]:
X_train_selecteddf = pd.DataFrame(X_train_selected)

# Coefficient plot (if applicable)
if hasattr(model, 'coef_'):
    coefs = pd.Series(model.coef_, index=X_train_selecteddf.columns)
    coefs.plot(kind='bar')
    plt.title('Feature Coefficients')
    plt.show()

### Testing linear regression 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Initialize logistic regression model
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)

# Fit the model on the training data
model_lr.fit(X_train_selected, y_train)

# Predict on the test data
y_pred_lr = model_lr.predict(X_test_selected)

# Evaluate model performance
print(classification_report(y_test, y_pred_lr))

In [ ]:
#confusion matrix
cm = confusion_matrix(y_test, y_pred_lr)
print(cm)


## Dimensionality Reduction
Here, we are testing a different way to reduce dimensionality by using Principal Component Analysis

In [ ]:
data = pd.read_csv("../data/raw/data.csv")
labels = pd.read_csv("../data/raw/labels.csv")
data.head()

X = data.drop(data.columns[0],axis=1)
Y = labels.Class

pca = PCA(n_components=5)
principalComponents = pca.fit_transform(X)
X_pca = pd.DataFrame(data = principalComponents)
X_pca = pd.concat([X_pca.reset_index().drop(['index'],axis=1),Y.reset_index().drop(['index'],axis=1)], axis=1)

sns.pairplot(x_vars=0, y_vars=1, data=X_pca, hue="Class",palette="YlGnBu",size=7,aspect=1.2)
plt.show()

In [ ]:
X = X_pca.drop(['Class'],axis=1)
Y = X_pca['Class']

X_train, X_test, Y_train, Y_test = train_test_split(X, y_encoded, test_size = 0.2, random_state = 42)

In [ ]:
# Initialize linear regression model
model = LinearRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict on the test data
y_pred_pca = model.predict(X_test)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred_pca)
r2 = r2_score(y_test, y_pred_pca)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

In [ ]:
residuals = y_test - y_pred_pca

plt.figure(figsize=(8, 6))
plt.scatter(y_pred_pca, residuals, color='blue', alpha=0.5)
plt.axhline(y=0, color='k', linestyle='--', lw=2)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

In [ ]:
# Distribution plot of residuals
plt.figure(figsize=(8, 6))
residuals = y_test - y_pred_pca
plt.hist(residuals, bins=20, color='blue', alpha=0.5)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Distribution of Residuals')
plt.show()